In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import numpy as np
import imgaug.augmenters as iaa
import tensorflow as tf
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


#/content/drive/My Drive/level_0_500
DATADIR = "/content/drive/MyDrive/Augmentation"
CATEGORIES = ["level0_aug","level4_aug"]

In [2]:
training_data = []


def create_training_data():
    for category in CATEGORIES :
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img),cv2.IMREAD_COLOR)[...,::-1]
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
               ## img_array = cv2.addWeighted(img_array,4, cv2.GaussianBlur(img_array , (0,0) , 20) ,-4 ,128)
                new_array = cv2.resize(img_array, (512, 512))
                training_data.append([new_array, class_num])
                rotate=iaa.Affine(rotate=(90))
                rotated_image=rotate.augment_image(new_array)
                training_data.append([rotated_image, class_num])
                flip_hr=iaa.Fliplr(p=1.0)
                flip_hr_image= flip_hr.augment_image(new_array)
                training_data.append([flip_hr_image, class_num])
            except Exception as e:
                pass

In [3]:
create_training_data()

In [4]:
random.shuffle(training_data)

In [5]:
X = [] #features
y = [] #labels

for features in training_data:
    X.append(features[0])
    y.append(features[1])
X = np.array(X)
X.shape

(627, 512, 512)

In [6]:
from sklearn.model_selection import train_test_split

train_X_full, test_X, train_y_full, test_y = train_test_split(X, y, test_size=0.2, stratify=y)

In [8]:
train_X_full = train_X_full/255.0
test_X = test_X/255.0

In [9]:
#create validation data as well, plus scaling down x values from 1-255 to 0-1 scale
X_valid, train_X = train_X_full[:60], train_X_full[60:]
y_valid, train_y = train_y_full[:60], train_y_full[60:]

In [10]:
X_valid.shape , train_X.shape,test_X.shape

((60, 512, 512), (441, 512, 512), (126, 512, 512))

In [11]:
X_valid = X_valid.reshape(60,262144)
train_X = train_X.reshape(441,262144)
test_X = test_X.reshape(126,262144)

In [12]:
X_valid.shape, train_X.shape,test_X.shape

((60, 262144), (441, 262144), (126, 262144))

In [13]:
from sklearn.neural_network import MLPClassifier

In [14]:
import numpy as np
train_y = np.array(train_y)

In [15]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(32,16,8,2),alpha= 0.01, batch_size= 10,learning_rate_init=0.0002,validation_fraction=0.01,
                     max_iter=100,activation = 'relu',
                       solver='sgd')
mlp_clf.fit(train_X, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.01, batch_size=10, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(32, 16, 8, 2), learning_rate='constant',
              learning_rate_init=0.0002, max_fun=15000, max_iter=100,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.01, verbose=False,
              warm_start=False)

In [16]:
test_y_pred = mlp_clf.predict(test_X)

In [17]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, test_y_pred)

array([[ 0, 63],
       [ 0, 63]])

In [18]:
from sklearn.metrics import accuracy_score

print(accuracy_score(test_y, test_y_pred))

0.5


In [19]:
from sklearn.metrics import classification_report
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.50      1.00      0.67        63

    accuracy                           0.50       126
   macro avg       0.25      0.50      0.33       126
weighted avg       0.25      0.50      0.33       126



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
